## Toxic comment classification
### Import library

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import log_loss,confusion_matrix,classification_report,roc_curve,auc

from sklearn.model_selection import train_test_split

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy import sparse
%matplotlib inline
seed = 42
import os
os.environ['OMP_NUM_THREADS'] = '4'

## Read data set

In [20]:
#labeled data
toxic = pd.read_csv('toxicity_data/train.csv')
#test = pd.read_csv('toxicity_data/test.csv') # this test dataset doesn't have labels b/c it is from kaggle, delete?
print('Number of rows and columns in the train data set:',toxic.shape)
#print('Number of rows and columns in the test data set:',test.shape)

#unlabeled data
incel_df = pd.read_csv('new_IncelTears_posts.csv')
slate_df = pd.read_csv('new_slatestarcodex_posts.csv')

raw_toxic = toxic
#small_toxic = toxic.sample(50000)
small_toxic = toxic


target_col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
y = small_toxic[target_col]
y['sum'] = y.sum(axis=1).astype(bool).astype(int)


X_train, X_test, y_train, y_test = train_test_split(small_toxic, y, test_size=0.2, random_state=42)

### delete? Not probably going to use test.
# raw_test = test
# test = test.sample(5000)
# test.fillna(' ',inplace=True)

Number of rows and columns in the train data set: (159571, 8)


/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Text preprosesing

[source: ](https://www.kaggle.com/him4318/easy-and-fast-lb-044) 

Term Frequency Inverse Document  Frequency Vectorizer 

In [21]:
vect_word = TfidfVectorizer(max_features=20000, lowercase=True, analyzer='word',
                        stop_words= 'english',ngram_range=(1,3),dtype=np.float32)
tr_vect = vect_word.fit_transform(X_train['comment_text'])
ts_vect = vect_word.transform(X_test['comment_text'])

incel_vect = vect_word.transform(incel_df['title'])
slate_vect = vect_word.transform(slate_df['title'])

#took 50 seconds on 150k samples

## LR Model

In [22]:
lr = LogisticRegression(C=2,random_state = 42,class_weight = 'balanced')
lr.fit(tr_vect,y_train['sum'])

col = 'insult'
print("Column:",col)
pred =  lr.predict(ts_vect)
print('\nConfusion matrix\n',confusion_matrix(y_test['sum'],pred))
print(classification_report(y_test['sum'],pred))

Column: insult

Confusion matrix
 [[27150  1521]
 [  491  2753]]
              precision    recall  f1-score   support

           0       0.98      0.95      0.96     28671
           1       0.64      0.85      0.73      3244

   micro avg       0.94      0.94      0.94     31915
   macro avg       0.81      0.90      0.85     31915
weighted avg       0.95      0.94      0.94     31915



In [23]:
from sklearn.metrics import f1_score
f1_score(y_test['sum'],pred)  

0.7323756318169726

## Take a look at the negative Incel Subreddit

In [24]:
incel_preds = lr.predict(incel_vect)
print(f'Percentage of Incel titles predicted as toxic {incel_preds.sum()/incel_preds.shape[0]}')

Percentage of Incel titles predicted as insults 0.3550651955867603


In [25]:
incel_df[np.isin(incel_preds, 0)]['title'].values #these are the ones it said were ok.

array(['Another one thinking he is a genious', '"But were nonviolent"',
       'Probably a LARP as I imagine the only thing he actually lifts is Cheeto packets. However if true, I hope the next woman be tries it with gives him what he deserves...',
       'Because of course he’s entitled to a woman’s body if someone else has had it.',
       'They bring so much of their unhappiness upon themselves',
       'incel worried about an epidemic of "open mouthed skinny framed" guys',
       '"clothes. But her bone structure is terrible, if she was born male, she would be extremely repulsive and for sure involuntary adult virgin"',
       '“Females are the problem, not males.”',
       'Incel goes outside and overhears a conversation that had nothing to do with him and takes personal offense.',
       'Incel blaming genetics while at the same time refusing to work on himself, you ain’t gonna get anywhere by doing that.',
       "Y'all know there's a new sub right?",
       'Incel discussing wo

Still some pretty bad stuff getting missed.

## Let's look at  a more supportive subreddit

In [26]:
slate_preds = lr.predict(slate_vect)
print(f'Percentage of Slate titles predicted as toxic {slate_preds.sum()/slate_preds.shape[0]}')

Percentage of Slate titles predicted as insults 0.0712136409227683


In [27]:
(incel_preds.sum()/incel_preds.shape[0])/(slate_preds.sum()/slate_preds.shape[0]) #good subreddit 13x better.

4.985915492957747